<a href="https://colab.research.google.com/github/TheTrappist/teaching/blob/main/Biochem6761/AU23/01_Intro_AU23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Hi, and welcome to Biochem 6761!
# This is a comment. The computer won't do anything about it.
# Single-line comments are indicated by a '#' sign.

"""
Multi-line comments look
like this!
"""

# Let's have the computer actually do something.



# Remove the comments below to run the sample code
"""
a = 5 # a is a variable. we can call it (almost) anything we want

print(a)
print(a+5)

# this is a simple loop:
for b in range(0,8):
  c = b*b
  print(c)
"""


In [ ]:
# Let's do some simple plotting!

# The following are import statements. They load additional
# functions stored in libraries
import numpy as np
from matplotlib import pyplot as plt

x = np.linspace(0, 10, 100)

# This is a definition of a function
def exponent(x,a):
  y = np.power(x,a)
  return(y)

y = exponent(x,2)

plt.style.use("bmh")

# Let's make a simple plot
fig, ax = plt.subplots()
ax.plot(x,y)

#print(plt.style.available)

**This is a text cell. It won't trigger any code to run and can be used to annotate (describe) segments of code.**

# This is a cell storing text and equations in LaTeX.

The simple hyperbolic binding formula assumes that the concentration of receptor is very low:

$f_{bound}=\frac{[L]_{total}}{[L]_{total}+K_D}$

\\

The fuller (and more accurate) quadratic form explicitly accoutns for the concentration of protein/receptor:

$f_{bound}=\frac{([R]_{total}+[L]_{total}+K_D)-\sqrt{([R]_{total}+[L]_{total}+K_D)^2-4\times[R]_{total}\times{[L]_{total}}}}{2\times[R]_{total}}$

\\

In both equations:

- $f_{bound}$ is the fraction of receptors that have a ligand bound
- $K_D$ is the dissociation constant for the ligand
- $[L]_{total}$ is the total concentration of the ligand in solution
- $[R]_{total}$ is the total concentration of protein (receptor) in solution

\\

A fun paper for reference that blasts most published binding analyses:https://elifesciences.org/articles/57264

In [ ]:
# Rewrite the functions described in the previous cell in actual code

def f_bound_hyperbolic(L_tot,K_D):
  f_bound = 1 / (1 + K_D/L_tot)
  return f_bound

def f_bound_quadratic(L_tot, K_D, R_tot):
  # recast into standard quadratic equation for clarity
  a = R_tot
  b = -(R_tot+L_tot+K_D)
  c = L_tot
  f_bound = (-b - np.sqrt(b**2-4*a*c)) / (2*a)
  return f_bound

# Define a concentration range and parameters.
L_tot = np.linspace(0.1,500,500)
K_D = 50
R_tot = 300

fig, axs = plt.subplots(2,1)

f_hyper = f_bound_hyperbolic(L_tot,K_D)
f_quad = f_bound_quadratic(L_tot, K_D, R_tot)

axs[0].plot(L_tot, f_hyper)
axs[1].plot(L_tot, f_quad)
axs[1].set_xlabel('$L_{tot}$')


# Can you make the quadratic plot for a wide range of R_tot values?

"""
Replace this with your code!
"""

In [ ]:
# Now, let's simulate some biding data and see if we can fit it

# Load the nonlinear least squares fit function:
from scipy.optimize import curve_fit


def simulate_quad_data(ligand_conc_array, R_tot, K_D, noise_std):
  f_bound_array = f_bound_quadratic(ligand_conc_array, K_D, R_tot)
  noise = np.random.normal(scale=noise_std, size = f_bound_array.size)
  result_with_noise = f_bound_array + noise
  return result_with_noise


# simulate data
sim_array = np.logspace(0.01,2.7,6)
noise_std = 0.03
sim_y_values = simulate_quad_data(sim_array, R_tot, K_D, noise_std)

# plot the simulated data
fig2, ax2 = plt.subplots()
ax2.plot(sim_array, sim_y_values, ls='', marker='o', color='g')

# fit the simulated data

R_tot = 300

# prepare function for fitting while keeping R_tot constant
def quad_with_R_tot(L_tot, K_D):
  return(f_bound_quadratic(L_tot, K_D, R_tot))

# this is the actual fitting step:
popt, pcov = curve_fit(quad_with_R_tot, sim_array, sim_y_values)

# print fit results
print("Kd from quadratic fit:", popt)

# plot fit results
f_quad_fit = f_bound_quadratic(L_tot, popt[0], R_tot)
ax2.plot(L_tot, f_quad_fit)

# does fitting with a hyperbolic function work as well?
popt, pcov = curve_fit(f_bound_hyperbolic, sim_array, sim_y_values)
f_hyper_fit = f_bound_hyperbolic(L_tot, popt[0])
ax2.plot(L_tot, f_hyper_fit)
print("Kd from hyperbolic fit:", popt)